# Title and Introduction TBD

<br>

## I. Preparation
***

- Install and import libraries

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from bayes_opt import BayesianOptimization

from sklearn.preprocessing import LabelEncoder
from sklearn.ensemble import RandomForestClassifier as rfc
from sklearn.model_selection import cross_val_score

import xgboost as xgb
from xgboost import XGBClassifier as xgbc, cv

import warnings

# Show all columns
pd.set_option('display.max_columns', None)

# Ignore all warnings
warnings.filterwarnings("ignore")

- Read datasets

In [2]:
df = pd.read_csv("../Data/airplane_train_processed_normalized.csv")
df_val = pd.read_csv("../Data/airplane_test_processed_normalized.csv")


df = df.head(1000)
df_val = df_val.head(1000)


#Removing the highly correlated columns like 'cat__Gender_Male','cat__Customer Type_disloyal Customer',
#'cat__Type of Travel_Personal Travel','cat__Class_Eco',
df = df[['Gender_Female', 'Customer Type_Loyal Customer',
         'Type of Travel_Business travel',
         'Type of Travel_Personal Travel', 'Class_Business',
         'Class_Eco', 'Age',
         'Flight Distance', 'Departure Delay in Minutes',
         'Arrival Delay in Minutes', 'Inflight wifi service',
         'Departure/Arrival time convenient',
         'Ease of Online booking', 'Gate location',
         'Food and drink', 'Online boarding',
         'Seat comfort', 'Inflight entertainment',
         'On-board service', 'Leg room service',
         'Baggage handling', 'Checkin service',
         'Inflight service', 'Cleanliness',
         'satisfaction']]

df.dropna(inplace=True)

In [3]:
X_train = df[['Gender_Female', 'Customer Type_Loyal Customer',
              'Type of Travel_Business travel',
              'Type of Travel_Personal Travel', 'Class_Business',
              'Class_Eco', 'Age',
              'Flight Distance', 'Departure Delay in Minutes',
              'Arrival Delay in Minutes', 'Inflight wifi service',
              'Departure/Arrival time convenient',
              'Ease of Online booking', 'Gate location',
              'Food and drink', 'Online boarding',
              'Seat comfort', 'Inflight entertainment',
              'On-board service', 'Leg room service',
              'Baggage handling', 'Checkin service',
              'Inflight service', 'Cleanliness',]]

y_train = df['satisfaction'].values

X_test = df_val[['Gender_Female', 'Customer Type_Loyal Customer',
                 'Type of Travel_Business travel',
                 'Type of Travel_Personal Travel', 'Class_Business',
                 'Class_Eco', 'Age',
                 'Flight Distance', 'Departure Delay in Minutes',
                 'Arrival Delay in Minutes', 'Inflight wifi service',
                 'Departure/Arrival time convenient',
                 'Ease of Online booking', 'Gate location',
                 'Food and drink', 'Online boarding',
                 'Seat comfort', 'Inflight entertainment',
                 'On-board service', 'Leg room service',
                 'Baggage handling', 'Checkin service',
                 'Inflight service', 'Cleanliness']]

y_test = df_val['satisfaction'].values

unique_categories, category_counts = np.unique(y_train, return_counts=True)
np.unique(y_train, return_counts=True)

(array(['neutral or dissatisfied', 'satisfied'], dtype=object),
 array([561, 439]))

In [4]:
X_train.head()

,Gender_Female,Customer Type_Loyal Customer,Type of Travel_Business travel,Type of Travel_Personal Travel,Class_Business,Class_Eco,Age,Flight Distance,Departure Delay in Minutes,Arrival Delay in Minutes,Inflight wifi service,Departure/Arrival time convenient,Ease of Online booking,Gate location,Food and drink,Online boarding,Seat comfort,Inflight entertainment,On-board service,Leg room service,Baggage handling,Checkin service,Inflight service,Cleanliness
0,0.0,1.0,0.0,1.0,0.0,0.0,-1.746468,6.133398,3.258097,2.944439,3.0,4.0,3.0,1.0,5.0,3.0,5.0,5.0,4.0,3.0,4.0,4.0,5.0,5.0
1,0.0,0.0,1.0,0.0,1.0,0.0,-0.951927,5.463832,0.693147,1.945910,3.0,2.0,3.0,3.0,1.0,3.0,1.0,1.0,1.0,5.0,3.0,1.0,4.0,1.0
2,1.0,1.0,1.0,0.0,1.0,0.0,-0.885715,7.041412,0.000000,0.000000,2.0,2.0,2.0,2.0,5.0,5.0,5.0,5.0,4.0,3.0,4.0,4.0,4.0,5.0
3,1.0,1.0,1.0,0.0,1.0,0.0,-0.951927,6.333280,2.484907,2.302585,2.0,5.0,5.0,5.0,2.0,2.0,2.0,2.0,2.0,5.0,3.0,1.0,4.0,2.0
4,0.0,1.0,1.0,0.0,1.0,0.0,1.431697,5.370638,0.000000,0.000000,3.0,3.0,3.0,3.0,4.0,5.0,5.0,3.0,3.0,4.0,4.0,3.0,3.0,3.0


<br>

- Set random seed and the number of folds

In [5]:
SEED = 42
CV = 4

<br>

## II. Bayesian Optimization on Random Forest Classifier
***

- Define the function to be optimized

In [64]:
# Define rfc_cv as Random Forest Classifier returning Cross Validation Score
def rfc_cv(n_estimators, min_samples_split, max_features, data, targets):
    estimator = rfc(n_estimators = n_estimators,
                    min_samples_split = min_samples_split,
                    max_features = max_features,
                    random_state = SEED)
    cval = cross_val_score(estimator, data, targets, scoring = 'accuracy', cv = CV)
    return cval.mean()

- Define parameter bounds

In [65]:
pbounds = {'max_features': (0.1, 0.999),
           'min_samples_split': (2, 25),
           'n_estimators': (10, 250)}

- Create an optimization function for random forest classifier

In [66]:
# Apply Bayesian Optimization on Random Forest parameters
def optimize_rfc(data, targets):
    # To ensure data type (float -> int) for rfc_cv inputs
    def rfc_crossval(n_estimators, min_samples_split, max_features):
        return rfc_cv(n_estimators = int(n_estimators),
                      min_samples_split = int(min_samples_split),
                      max_features = max(min(max_features, 0.999), 1e-3),
                      data = data,
                      targets = targets)
    
    optimizer = BayesianOptimization(f = rfc_crossval,
                                     pbounds = pbounds,
                                     random_state = SEED,
                                     verbose = 2)
    optimizer.maximize(n_iter = 10, init_points = 2)
    
    # Return the best result, and results from each iteration
    return {"max": optimizer.max, "res": optimizer.res}

In [67]:
opt_rfc = optimize_rfc(X_train, y_train)

|   iter    |  target   | max_fe... | min_sa... | n_esti... |
-------------------------------------------------------------
| 1         | 0.898     | 0.4367    | 23.87     | 185.7     |
| 2         | 0.904     | 0.6382    | 5.588     | 47.44     |
| 3         | 0.914     | 0.2159    | 4.945     | 46.46     |
| 4         | 0.904     | 0.2445    | 11.18     | 30.76     |
| 5         | 0.907     | 0.3924    | 13.02     | 44.27     |
| 6         | 0.903     | 0.9477    | 10.46     | 63.11     |
| 7         | 0.892     | 0.9038    | 21.81     | 109.2     |
| 8         | 0.91      | 0.7293    | 5.515     | 46.05     |
| 9         | 0.914     | 0.1945    | 3.744     | 45.77     |
| 10        | 0.913     | 0.1541    | 4.799     | 44.34     |
| 11        | 0.908     | 0.5556    | 2.443     | 43.38     |
| 12        | 0.913     | 0.7329    | 5.669     | 41.76     |


In [68]:
print(opt_rfc["max"])

{'target': 0.914, 'params': {'max_features': 0.21586286999767368, 'min_samples_split': 4.945054296028767, 'n_estimators': 46.45664644295064}}


In [70]:
iterations = {}

cols = ["iteration", "target"]
cols.extend(list(pbounds.keys()))

for col in cols:
    iterations[col] = []

iterations = pd.DataFrame(iterations)

for i, res in enumerate(opt_rfc["res"]):
    iterations.loc[i] = [str(i), 
                         res["target"], 
                         max(min(res["params"]["max_features"], 0.999), 1e-3),
                         round(res["params"]["min_samples_split"]), 
                         round(res["params"]["n_estimators"])]
    
display(iterations)

,iteration,target,max_features,min_samples_split,n_estimators
0,0,0.898,0.436712,24,186
1,1,0.904,0.638194,6,47
2,2,0.914,0.215863,5,46
3,3,0.904,0.244545,11,31
4,4,0.907,0.392384,13,44
5,5,0.903,0.947749,10,63
6,6,0.892,0.903793,22,109
7,7,0.910,0.729259,6,46
8,8,0.914,0.194518,4,46
9,9,0.913,0.154141,5,44


<br>

## III. Bayesian Optimization on XGB Classifier
***

- Define the function to be optimized

In [71]:
# Define xgb_cv as XGB Classifier returning Cross Validation Score
def xgb_cv(max_depth, alpha, learning_rate, data, targets):
    le = LabelEncoder()
    target_ = le.fit_transform(targets)
    data_dmatrix = xgb.DMatrix(data = data, label = target_)

    params = {'objective':'binary:logistic',
              'max_depth': max_depth,
              'alpha': alpha,
              'learning_rate': learning_rate}

    xgb_cv_score = cv(dtrain = data_dmatrix, 
                      params = params, 
                      nfold = CV,
                      num_boost_round = 50, 
                      early_stopping_rounds = 10, 
                      metrics = "auc", 
                      as_pandas = True, 
                      seed = SEED)

    return xgb_cv_score["test-auc-mean"].mean()

- Define parameter bounds

In [72]:
pbounds = {'alpha': (1, 180), 
           'learning_rate': (0.01, 1.0), 
           'max_depth': (10, 250)}

- Create an optimization function for XGB classifier

In [73]:
# Apply Bayesian Optimization on Random Forest parameters
def optimize_xgb(data, targets):
    # To ensure data type (float -> int) for xgb_cv inputs
    def xgb_crossval(max_depth, alpha, learning_rate):
        return xgb_cv(max_depth = int(max_depth),
                      alpha = int(alpha),
                      learning_rate = learning_rate,
                      data = data,
                      targets = targets)
    
    optimizer = BayesianOptimization(f = xgb_crossval,
                                     pbounds = pbounds,
                                     random_state = SEED,
                                     verbose = 2)
    
    optimizer.maximize(n_iter = 10, init_points = 2)
    
    # Return the best result, and results from each iteration
    return {"max": optimizer.max, "res": optimizer.res}

In [74]:
opt_xgb = optimize_xgb(X_train, y_train)

|   iter    |  target   |   alpha   | learni... | max_depth |
-------------------------------------------------------------
| 1         | 0.8872    | 68.04     | 0.9512    | 185.7     |
| 2         | 0.8211    | 108.2     | 0.1645    | 47.44     |
| 3         | 0.8992    | 67.55     | 0.2517    | 185.0     |
| 4         | 0.8747    | 64.6      | 0.01      | 180.6     |
| 5         | 0.8924    | 69.52     | 0.6006    | 181.4     |
| 6         | 0.9016    | 61.57     | 0.2831    | 187.7     |
| 7         | 0.9062    | 57.04     | 0.3135    | 189.2     |
| 8         | 0.9017    | 59.9      | 0.4798    | 193.5     |
| 9         | 0.9139    | 52.41     | 0.2922    | 194.8     |
| 10        | 0.9137    | 51.07     | 0.6984    | 189.7     |
| 11        | 0.919     | 45.97     | 0.1471    | 193.6     |
| 12        | 0.9186    | 44.42     | 0.1302    | 187.8     |


In [75]:
print(opt_xgb["max"])

{'target': 0.9190325048697815, 'params': {'alpha': 45.967576791022296, 'learning_rate': 0.14712842812756818, 'max_depth': 193.63090589918684}}


In [78]:
iterations = {}

cols = ["iteration", "target"]
cols.extend(list(pbounds.keys()))

for col in cols:
    iterations[col] = []

iterations = pd.DataFrame(iterations)

for i, res in enumerate(opt_xgb["res"]):
    iterations.loc[i] = [str(i), 
                         res["target"], 
                         round(res["params"]["alpha"]), 
                         res["params"]["learning_rate"], 
                         round(int(res["params"]["max_depth"]))]
    
display(iterations)

,iteration,target,alpha,learning_rate,max_depth
0,0,0.887209,68,0.951207,185
1,1,0.821119,108,0.164458,47
2,2,0.899234,68,0.251654,184
3,3,0.874678,65,0.010000,180
4,4,0.892378,70,0.600593,181
5,5,0.901632,62,0.283100,187
6,6,0.906223,57,0.313514,189
7,7,0.901655,60,0.479798,193
8,8,0.913858,52,0.292192,194
9,9,0.913653,51,0.698400,189
